In [ ]:
!pip install accelerate -U
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 7.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments
import torch

In [ ]:

data = pd.read_csv("Sheet1.csv")

In [ ]:
data.head()

,question,query
0,Insert a new document with the fields 'name' a...,"db.contacts.insertOne({ name: 'Jane', phone: '..."
1,Find all documents where the 'quantity' field ...,db.documents.find({ quantity: { $ne: 0 } })
2,Retrieve all users who have not logged in for ...,db.users.find({ last_login: { $lt: new Date(Da...
3,Find all products with a price between $10 and...,"db.products.find({ price: { $gt: 10, $lt: 50 } })"
4,Get the titles of all articles published in th...,db.articles.find({ publication_date: { $gte: n...


In [ ]:
def preprocess_data(data):
    data['input_text'] = "question: " + data['question']
    data['target_text'] = data['query']
    return data

data = preprocess_data(data)

# Split the data into training and validation sets
train_data, val_data = train_test_split(data, test_size=0.1)

# Tokenization
tokenizer = T5Tokenizer.from_pretrained('t5-small')

def tokenize(batch):
    tokenized_input = tokenizer(batch['input_text'], padding='max_length', truncation=True, max_length=512)
    tokenized_target = tokenizer(batch['target_text'], padding='max_length', truncation=True, max_length=512)

    return {
        'input_ids': tokenized_input.input_ids,
        'attention_mask': tokenized_input.attention_mask,
        'labels': tokenized_target.input_ids
    }

train_data = train_data.apply(tokenize, axis=1)
val_data = val_data.apply(tokenize, axis=1)

class T5Dataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        item = self.data.iloc[index]
        return {key: torch.tensor(val) for key, val in item.items()}

train_dataset = T5Dataset(train_data)
val_dataset = T5Dataset(val_data)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# Model
model = T5ForConditionalGeneration.from_pretrained('t5-small')

# Training
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=50,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()

# Save the model
model.save_pretrained('./T5_fine_tuned')
tokenizer.save_pretrained('./t5_fine_tuned')


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss
10,16.439200
20,16.711500
30,16.380200
40,15.022600
50,14.897200
60,13.790400
70,12.653500
80,10.126900
90,9.278900
100,7.137700


('./t5_fine_tuned/tokenizer_config.json',
 './t5_fine_tuned/special_tokens_map.json',
 './t5_fine_tuned/spiece.model',
 './t5_fine_tuned/added_tokens.json')

In [ ]:
# Load the trained model and tokenizer
model_path = './T5_fine_tuned'
tokenizer_path = './t5_fine_tuned'
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained(tokenizer_path)

def generate_query(question, model, tokenizer):
    # Prepend 'question:' to the input question
    input_text = "question: " + question
    # Encode the input text
    input_ids = tokenizer.encode(input_text, return_tensors="pt", add_special_tokens=True)
    # Generate the output (query)
    output_ids = model.generate(input_ids, max_length=512, num_beams=5, early_stopping=True)
    # Decode and return the generated text
    query = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return query

OSError: Incorrect path_or_model_id: './T5_fine_tuned'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [ ]:
question = "Insert a new document with the fields 'username' and 'age' into the 'customers' collection."
query = generate_query(question, model, tokenizer)
print(query)

In [ ]:
from google.colab import drive
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Mount Google Drive
drive.mount('/content/drive')

# Path to save the model in Google Drive
drive_path = "/content/drive/My Drive/saved_model"

# Load the trained model and tokenizer
model_path = './T5_fine_tuned'
tokenizer_path = './t5_fine_tuned'
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained(tokenizer_path)

# Save the model and tokenizer to Google Drive
model.save_pretrained(drive_path)
tokenizer.save_pretrained(drive_path)

print("Model saved to Google Drive.")

Mounted at /content/drive


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Model saved to Google Drive.


In [ ]:
from google.colab import drive
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load the saved model and tokenizer
model_path = "/content/drive/My Drive/saved_model"
tokenizer_path = "/content/drive/My Drive/saved_model"
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained(tokenizer_path)

def generate_query(question, model, tokenizer):
    # Prepend 'question:' to the input question
    input_text = "question: " + question
    # Encode the input text
    input_ids = tokenizer.encode(input_text, return_tensors="pt", add_special_tokens=True)
    # Generate the output (query)
    output_ids = model.generate(input_ids, max_length=512, num_beams=5, early_stopping=True)
    # Decode and return the generated text
    query = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return query

# Example usage
question = "find all documents from users"
query = generate_query(question, model, tokenizer)
print("Generated query:", query)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Generated query: db.documents.find( $and: [  $gte: new Date(Date.now() - 30 * 24 * 60 * 60 * 1000)  )
